In [2]:
#Calculate next-generation matrix for three age-classes.

def giveNGM(R0):
    contactMatrix = np.array([[9,3,1],[3,3,2],[1,2,2]])      #Contact matrix between young, middle, and old.
    demography = givePopulationDemocraphy()

    eigenvalues, eigenvectors = np.linalg.eig(demography*contactMatrix)
    dominantEigenvalue = eigenvalues[0]
    
    q = R0/dominantEigenvalue           #The scaling parameter such that dominant eigenvalue of q*R is R0.

    return q*contactMatrix, q*demography*contactMatrix

def givePopulationDemocraphy():
    return np.array([1/8.,5/8.,2/8.])

In [3]:
def finalSizeFunction(NGM,s0,z):
    return np.array([s*(1.0 - np.exp(-sum(NGM[i,:]*z))) for i,s in enumerate(s0)])

def findFinalSize(NGM,s0):
    
    zOld = finalSizeFunction(NGM,s0,np.ones(3))
    
    stepChange = 1
    while(stepChange > 0.00000001):
        zNew = finalSizeFunction(NGM,s0,zOld)
        
        stepChange = abs(sum(zOld-zNew))
        zOld = zNew
    
    return zNew

In [9]:
def giveNextSusceptibility(parameters,sIn):
    R0 = parameters['R0']
    gamma = parameters['gamma']
    delta = parameters['delta']
    Vc = parameters['Vc']
    Ve = parameters['Ve']
    NGM = parameters['NGM']
    
    #First vaccination, then epidemic, at last waning.
    
    s1 = (1.0-Vc*Ve)*sIn
    
    z = findFinalSize(NGM,s1)
    
    s2 = s1 - z
    s3 = np.ones(3) - gamma*( np.ones(3) - s2 )
    sOut = (1.0-delta)*s3 + delta*np.array([1,s3[0],s3[1]])

    #Return finalsizes and susceptibilities.
    return z, sOut

In [11]:
#Timeseries functions.
import random as rnd
import numpy as np

def calculateTimeseries(parameters,years,mode):
    
    #Add burn-in period.
    years = range(0,len(years)+50)
    
    #Initialise output vectors.
    timeseriesSusc = [ np.ones(3) ]         #Fraction susceptibles at start of season.
    timeseriesFinal = [ np.zeros(3) ]       #Fraction infected during seasonal epidemic.
    
    for year in years:

        if mode == 'Stochastic':
            parameters['gamma'] = rnd.betavariate(5,2)
        elif mode != 'Deterministic':
            print 'Wrong mode. Set mode to Deterministic or Stochastic.'
        
        s = timeseriesSusc[-1]
        
        finalSize, susceptibility = giveNextSusceptibility(parameters,s)
        
        timeseriesSusc += [ susceptibility ]
        timeseriesFinal += [ finalSize ]

    #Remove burn-in period.
    return np.array(timeseriesSusc[51:]), np.array(timeseriesFinal[51:])

In [ ]:
#SIR-model for one season, with three age structures.
import numpy as np
from scipy.integrate import odeint

def SIRage(R0,s0,smallPopulation=False):
    nu = 0.5
    beta = nu*R0
    N = 1000000.0
    stepsSeason = 1000
    
    #Get population distribution and next-generation matrix (of the total population).
    frequencies = givePopulationDemocraphy()
    PopulationNGM, StratifiedNGM = giveNGM(R0)

    if sum(N*s0) < 100:
        print 'Warning: N to small'
        
    def derivative(y,t):
        #S=y[0:3], I=y[3:6], R=y[6:9]
        
        dSdt = -(nu/N)*y[0:3]*np.dot(PopulationNGM,y[3:6])
        dIdt = (nu/N)*y[0:3]*np.dot(PopulationNGM,y[3:6]) - nu*y[3:6]
        dRdt = nu*y[3:6]
        
        return np.concatenate([dSdt, dIdt, dRdt])

    endTime = 1000
    time = np.linspace(0.0,endTime,stepsSeason)
    
    Sinit = (s0-0.00001)*frequencies*N
    Iinit = 0.00001*frequencies*N
    Rinit = (1.0-s0)*frequencies*N
    
    yinit = np.concatenate([Sinit,Iinit,Rinit])

    y = odeint(derivative,yinit,time)
    
    return time, y/N

In [ ]:
#Peak incidence function.

def peakIncidence(R0,s0):
    time, y = SIRage(R0,s0,smallPopulation=True)
    return max(np.sum(y[:,3:6],axis=1))

In [ ]:
#Functions below are the same as in Functions of the main folder (all-or-nothing vaccine).

In [ ]:
#Plot function. Removes box around plot.
def removeFrame(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    for tic in ax.xaxis.get_major_ticks():
        tic.tick2On = False
    for tic in ax.yaxis.get_major_ticks():
        tic.tick2On = False

In [ ]:
#Save data to file for later import back into python.
import pickle

def writeData(filename,data):
    outfile = open(filename + '.dat', 'w' )
    pickle.dump(data, outfile)
    outfile.close()

#Load data again.
def readData(filename):
    infile = open(filename + '.dat', 'r')
    return pickle.load(infile)